In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import KFold, cross_val_score

from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
uploaded = files.upload()

Saving credit-g_csv.csv to credit-g_csv.csv


In [3]:
df = pd.read_csv("credit-g_csv.csv")
df

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,4,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,2,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,3,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,4,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,4,no known property,53,none,for free,2,skilled,2,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12,existing paid,furniture/equipment,1736,<100,4<=X<7,3,female div/dep/mar,none,4,real estate,31,none,own,1,unskilled resident,1,none,yes,good
996,<0,30,existing paid,used car,3857,<100,1<=X<4,4,male div/sep,none,4,life insurance,40,none,own,1,high qualif/self emp/mgmt,1,yes,yes,good
997,no checking,12,existing paid,radio/tv,804,<100,>=7,4,male single,none,4,car,38,none,own,1,skilled,1,none,yes,good
998,<0,45,existing paid,radio/tv,1845,<100,1<=X<4,4,male single,none,4,no known property,23,none,for free,1,skilled,1,yes,yes,bad


# Исследование данных

In [4]:
# размерность датафрейма (первое число - кол-во строк, второе - колонок)
print(df.shape, "\n________________________________\n")

# информация о столбцах датафрейма (номер столбца, название колонки, количество непустых ячеек в столбце, тип данных в ячейках)
print(df.info(), "\n________________________________\n")

# с помощью метода isna() и sum() посчитали сколько пропусков в ячейках колонок (isna() "ищет" эти ячейки, sum() считает их суммарное количество)
print(df.isna().sum())

(1000, 21) 
________________________________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   checking_status         1000 non-null   object
 1   duration                1000 non-null   int64 
 2   credit_history          1000 non-null   object
 3   purpose                 1000 non-null   object
 4   credit_amount           1000 non-null   int64 
 5   savings_status          1000 non-null   object
 6   employment              1000 non-null   object
 7   installment_commitment  1000 non-null   int64 
 8   personal_status         1000 non-null   object
 9   other_parties           1000 non-null   object
 10  residence_since         1000 non-null   int64 
 11  property_magnitude      1000 non-null   object
 12  age                     1000 non-null   int64 
 13  other_payment_plans     1000 non-null   object
 14  housing    

In [5]:
# колонки checking_status, duration, purpose, personal_status, own_telephone, credit_amount, age не несут смысловой нагрузки, поэтому их можно удалить с помощью метода drop()
df = df.drop(['checking_status', 'duration', 'purpose', 'personal_status', 'own_telephone', 'credit_amount', 'age'], axis = 1)

In [6]:
df

,credit_history,savings_status,employment,installment_commitment,other_parties,residence_since,property_magnitude,other_payment_plans,housing,existing_credits,job,num_dependents,foreign_worker,class
0,critical/other existing credit,no known savings,>=7,4,none,4,real estate,none,own,2,skilled,1,yes,good
1,existing paid,<100,1<=X<4,2,none,2,real estate,none,own,1,skilled,1,yes,bad
2,critical/other existing credit,<100,4<=X<7,2,none,3,real estate,none,own,1,unskilled resident,2,yes,good
3,existing paid,<100,4<=X<7,2,guarantor,4,life insurance,none,for free,1,skilled,2,yes,good
4,delayed previously,<100,1<=X<4,3,none,4,no known property,none,for free,2,skilled,2,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,existing paid,<100,4<=X<7,3,none,4,real estate,none,own,1,unskilled resident,1,yes,good
996,existing paid,<100,1<=X<4,4,none,4,life insurance,none,own,1,high qualif/self emp/mgmt,1,yes,good
997,existing paid,<100,>=7,4,none,4,car,none,own,1,skilled,1,yes,good
998,existing paid,<100,1<=X<4,4,none,4,no known property,none,for free,1,skilled,1,yes,bad


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   credit_history          1000 non-null   object
 1   savings_status          1000 non-null   object
 2   employment              1000 non-null   object
 3   installment_commitment  1000 non-null   int64 
 4   other_parties           1000 non-null   object
 5   residence_since         1000 non-null   int64 
 6   property_magnitude      1000 non-null   object
 7   other_payment_plans     1000 non-null   object
 8   housing                 1000 non-null   object
 9   existing_credits        1000 non-null   int64 
 10  job                     1000 non-null   object
 11  num_dependents          1000 non-null   int64 
 12  foreign_worker          1000 non-null   object
 13  class                   1000 non-null   object
dtypes: int64(4), object(10)
memory usage: 109.5+ KB


Значений nan в датасете нет, поэтому можно сразу переходить к работе с категориальными признаками

# Преобразование категориальных признаков

In [ ]:
# категориальные признаки находящиеся в колонках credit_history, savings_status, employment, other_parties, 
# property_magnitude, other_payment_plans, housing, job, foreign_worker преобразуем с помощью метода get_dammies()

df = pd.get_dummies(df, columns=['credit_history', 'savings_status','employment', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'foreign_worker'])

# так как колонка class - это ответы, которые нужно представить в виде одной колонки, их преобразуем с помощью метода LabelEncoder 
x = LabelEncoder()
label = x.fit_transform(df['class'])
df['class'] = label
df

In [10]:
# создаём вектор признаков и вектор ответов
X = df.drop(['class'], axis=1)
y = df['class']

In [12]:
# для дальнейшей работы стандартизиуем и нормализуем данные с помощью функций StandardScaler и MinMaxScaler соответственно

# стандартизация
standart = StandardScaler()
X_st = standart.fit_transform(X)

# нормализация
norm = MinMaxScaler()
X_norm = norm.fit_transform(X)

# Дерево решений

## Обычные данные

In [14]:
# делим данными на тестовые и тренировочные с помощью метода train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

# создаем модель
tree = DecisionTreeClassifier(random_state=1)

# обучаем модель
tree.fit(X_train, y_train)

# тестируем модель
y_predict_tree = tree.predict(X_test)

# оценивать модель будем с помощью оценок accuracy, recall, precision

tree_accuracy = accuracy_score(y_test,y_predict_tree)
tree_recall =  recall_score(y_test,y_predict_tree)
tree_precision =  precision_score(y_test,y_predict_tree)

print("Tree accuracy:", tree_accuracy)
print("Tree recall:", tree_recall)
print("Tree precision:", tree_precision)

# оценки модели вышли средними (не очень хорошими, но и не плохими)

# матрица ошибок модели (метод confusion_matrix)

print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predict_tree))

DecisionTreeClassifier accuracy: 0.6287425149700598
DecisionTreeClassifier recall: 0.6982758620689655
DecisionTreeClassifier precision: 0.75

Confusion matrix:
 [[ 48  54]
 [ 70 162]]


## Нормализованные данные

In [ ]:
# делим данными на тестовые иы тренировочные
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

# создаем модель
tree = DecisionTreeClassifier(random_state=1)

# обучаем модель
tree.fit(X_train, y_train)

# тестируем модель
y_predict_tree_norm = tree.predict(X_test)

# оценка

tree_accuracy_norm = accuracy_score(y_test, y_predict_tree_norm)
tree_recall_norm =  recall_score(y_test, y_predict_tree_norm)
tree_precision_norm =  precision_score(y_test, y_predict_tree_norm)

print("Tree accuracy norm:", tree_accuracy_norm)
print("Tree recall norm:", tree_recall_norm)
print("Tree precision norm:", tree_precision_norm)

# оценки модели на нормализованных данных также вышли средними

# матрица ошибок модели

print('\nConfusion matrix norm:\n', confusion_matrix(y_test, y_predict_tree_norm))

DecisionTreeClassifier accuracy norm: 0.6197604790419161
DecisionTreeClassifier recall norm: 0.6982758620689655
DecisionTreeClassifier precision norm: 0.7397260273972602

Confusion matrix norm:
 [[ 45  57]
 [ 70 162]]


## Стандартизованные данные

In [ ]:
# делим данными на тестовые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

# создаем модель
tree = DecisionTreeClassifier(random_state=1)

# обучаем модель
tree.fit(X_train, y_train)

# тестируем модель
y_predict_tree_st = tree.predict(X_test)

# оценка
tree_accuracy_st = accuracy_score(y_test, y_predict_tree_st)
tree_recall_st =  recall_score(y_test, y_predict_tree_st)
tree_precision_st =  precision_score(y_test, y_predict_tree_st)

print("Tree accuracy st:", tree_accuracy_st)
print("Tree recall st:", tree_recall_st)
print("Tree precision st:", tree_precision_st)

# оценки модели на стандартизированных данных также вышли средними

# матрица ошибок

print('\nConfusion matrix st:\n', confusion_matrix(y_test,y_predict_tree_st))

DecisionTreeClassifier accuracy norm: 0.6287425149700598
DecisionTreeClassifier recall norm: 0.6982758620689655
DecisionTreeClassifier precision norm: 0.75

Confusion matrix st:
 [[ 48  54]
 [ 70 162]]


## Кроссвалидация

In [ ]:
kf = KFold()

# создаем модель
tree = DecisionTreeClassifier(random_state=1)

# оценка
tree_crs_accuracy = cross_val_score(estimator=tree, X=X, y=y, cv=kf, scoring='accuracy').mean()
tree_crs_recall = cross_val_score(estimator=tree, X=X, y=y, cv=kf, scoring='recall').mean()
tree_crs_precision = cross_val_score(estimator=tree, X=X, y=y, cv=kf, scoring='precision').mean()

# оценки модели с кроссвалидацией данных также вышли средними
print('Tree cross_val_score accuracy: ', tree_crs_accuracy)
print('Tree cross_val_score recall: ', tree_crs_recall)
print('Tree cross_val_score precision: ', tree_crs_precision)

DecisionTreeClassifier cross_val_score accuracy:  0.615
DecisionTreeClassifier cross_val_score recall:  0.7074126351388916
DecisionTreeClassifier cross_val_score precision:  0.7347245893478509


# Метод опорных векторов

## Обычные данные

In [ ]:
# делим данными на тестовые и тренировочные с помощью метода train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

# создаем модель
svm = SVC(kernel='linear', random_state=1)

# обучаем модель
svm.fit(X_train, y_train)

# тестируем модель
y_predict_svm = svm.predict(X_test)

# оценка

svm_accuracy = accuracy_score(y_test,y_predict_svm)
svm_recall =  recall_score(y_test,y_predict_svm)
svm_precision =  precision_score(y_test,y_predict_svm)

print("SVM accuracy:", svm_accuracy)
print("SVM recall:", svm_recall)
print("SVM precision:", svm_precision)

# модель показала хорошие результаты  

# матрица ошибок модели

print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predict_svm))

SVM accuracy: 0.7245508982035929
SVM recall: 0.9612068965517241
SVM precision: 0.7287581699346405

Confusion matrix:
 [[ 19  83]
 [  9 223]]


## Нормализованные данные

In [ ]:
# делим данными на тестовые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

# создаем модель
svm = SVC(kernel='linear', random_state=1)

# обучаем модель
svm.fit(X_train, y_train)

# тестируем модель
y_predict_svm_norm = svm.predict(X_test)

# оценка
svm_accuracy_norm = accuracy_score(y_test,y_predict_svm_norm)
svm_recall_norm =  recall_score(y_test,y_predict_svm_norm)
svm_precision_norm =  precision_score(y_test,y_predict_svm_norm)

print("SVM accuracy norm:", svm_accuracy_norm)
print("SVM recall norm:", svm_recall_norm)
print("SVM precision norm:", svm_precision_norm)

# модель показала абсолютно такие же рузельтаты, как и модель, построенная на обычных данных

# матрица ошибок модели

print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predict_svm_norm))

SVM accuracy norm: 0.7245508982035929
SVM recall norm: 0.9612068965517241
SVM precision norm: 0.7287581699346405

Confusion matrix:
 [[ 19  83]
 [  9 223]]


## Стандартизованные данные

In [ ]:
# делим данными на тестовые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

# создаем модель
svm = SVC(kernel='linear', random_state=1)

# обучаем модель
svm.fit(X_train, y_train)

# тестируем модель
y_predict_svm_st = svm.predict(X_test)

# оценка
svm_accuracy_st = accuracy_score(y_test,y_predict_svm_st)
svm_recall_st =  recall_score(y_test,y_predict_svm_st)
svm_precision_st =  precision_score(y_test,y_predict_svm_st)

print("SVM accuracy st:", svm_accuracy_st)
print("SVM recall st:", svm_recall_st)
print("SVM precision st:", svm_precision_st)

# модель показала абсолютно такие же рузельтаты, как и модель, построенная на обычных и нормализованных данных

# матрица ошибок модели

print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predict_svm_st))

SVM accuracy st: 0.7245508982035929
SVM recall st: 0.9612068965517241
SVM precision st: 0.7287581699346405

Confusion matrix:
 [[ 19  83]
 [  9 223]]


## Кроссвалидация

In [ ]:
kf = KFold()

# создаем модель
svm = SVC(kernel='linear', random_state=1)

# оценка
svm_crs_accuracy = cross_val_score(estimator=svm, X=X, y=y, cv=kf, scoring='accuracy').mean()
svm_crs_recall = cross_val_score(estimator=svm, X=X, y=y, cv=kf, scoring='recall').mean()
svm_crs_precision = cross_val_score(estimator=svm, X=X, y=y, cv=kf, scoring='precision').mean()

# оценки модели с кроссвалидацией данных также вышли средними
print('SVM cross_val_score accuracy: ', svm_crs_accuracy)
print('SVM cross_val_score recall: ', svm_crs_recall)
print('SVM cross_val_score precision: ', svm_crs_precision)

SVM cross_val_score accuracy:  0.717
SVM cross_val_score recall:  0.9487915402653291
SVM cross_val_score precision:  0.7287114941991099


# Логистическая регрессия

## Обычные данные

In [ ]:
# делим данными на тестовые и тренировочные с помощью метода train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

# создаем модель
linear = LogisticRegression()

# обучаем модель
linear.fit(X_train, y_train)

# тестируем модель
y_predicted_linear = linear.predict(X_test)

# оценка
linear_accuracy = accuracy_score(y_test,y_predicted_linear)
linear_recall =  recall_score(y_test,y_predicted_linear)
linear_precision =  precision_score(y_test,y_predicted_linear)

print("Logistic regression accuracy:", linear_accuracy)
print("Logistic regression recall:", linear_recall)
print("Logistic regression precision:", linear_precision)

# модель показала хорошие результаты

# матрица ошибок модели
print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predicted_linear))

LogisticRegression accuracy: 0.7544910179640718
LogisticRegression recall: 0.9482758620689655
LogisticRegression precision: 0.7586206896551724

Confusion matrix:
 [[ 32  70]
 [ 12 220]]


## Нормализованные данные

In [ ]:
# делим данными на тестовые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

# создаем модель
linear = LogisticRegression()

# обучаем модель
linear.fit(X_train, y_train)

# тестируем модель
y_predicted_linear_norm = linear.predict(X_test)

# оценка
linear_accuracy_norm = accuracy_score(y_test,y_predicted_linear_norm)
linear_recall_norm =  recall_score(y_test,y_predicted_linear_norm)
linear_precision_norm =  precision_score(y_test,y_predicted_linear_norm)

print("Logistic regression accuracy:", linear_accuracy_norm)
print("Logistic regression recall:", linear_recall_norm)
print("Logistic regression precision:", linear_precision_norm)

# модель показала абсолютно такие же результаты, как и модель, построенная на обычных данных

# матрица ошибок модели
print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predicted_linear_norm))

LogisticRegression accuracy: 0.7544910179640718
LogisticRegression recall: 0.9568965517241379
LogisticRegression precision: 0.7551020408163265

Confusion matrix:
 [[ 30  72]
 [ 10 222]]


## Стандартизованные данные

In [ ]:
# делим данными на тестовые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

# создаем модель
linear = LogisticRegression()

# обучаем модель
linear.fit(X_train, y_train)

# тестируем модель
y_predicted_linear_st = linear.predict(X_test)

# оценка
linear_accuracy_st = accuracy_score(y_test,y_predicted_linear_st)
linear_recall_st =  recall_score(y_test,y_predicted_linear_st)
linear_precision_st =  precision_score(y_test,y_predicted_linear_st)

# модель показала абсолютно такие же результаты, как и модель, построенная на обычных и нормализованных данных
print("Logistic regression accuracy:", linear_accuracy_st)
print("Logistic regression recall:", linear_recall_st)
print("Logistic regression precision:", linear_precision_st)

# матрица ошибок модели
print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predicted_linear_st))

LogisticRegression accuracy: 0.7574850299401198
LogisticRegression recall: 0.9439655172413793
LogisticRegression precision: 0.7630662020905923

Confusion matrix:
 [[ 34  68]
 [ 13 219]]


## Кроссвалидация

In [ ]:
kf = KFold()

# создаем модель
linear = LogisticRegression()

# оценка
linear_crs_accuracy_cvs = cross_val_score(estimator=linear, X=X, y=y, cv=kf, scoring='accuracy').mean()
linear_crs_recall_cvs = cross_val_score(estimator=linear, X=X, y=y, cv=kf, scoring='recall').mean()
linear_crs_precision_cvs = cross_val_score(estimator=linear, X=X, y=y, cv=kf, scoring='precision').mean()

# результат кроссвалидации вышел хуже, чем в остальных случаях логистической регрессии
print('Logistic regression cross_val_score accuracy: ', linear_crs_accuracy)
print('Logistic regression cross_val_score recall: ', linear_crs_recall)
print('Logistic regression cross_val_score precision: ', linear_crs_precision)

LogisticRegression cross_val_score accuracy:  0.72
LogisticRegression cross_val_score recall:  0.9143188652469545
LogisticRegression cross_val_score precision:  0.744321395205939


# KNN

## Обычные данные

In [ ]:
# делим данными на тестовые и тренировочные с помощью метода train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

# создаем модель, подбираем количество ближайших соседей методом Тыка
knn = KNeighborsClassifier(n_neighbors = 20) 

# обучаем модель
knn.fit(X_train, y_train)

# тестируем модель
y_predicted_knn = knn.predict(X_test)

# оценка
knn_accuracy = accuracy_score(y_test,y_predicted_knn)
knn_recall =  recall_score(y_test,y_predicted_knn)
knn_precision =  precision_score(y_test,y_predicted_knn)

print("KNN accuracy:", knn_accuracy)
print("KNN recall:", knn_recall)
print("KNN precision:", knn_precision)

# модель показала хороший результат

# матрица ошибок модели
print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predicted_knn))

KNeighborsClassifier accuracy: 0.7305389221556886
KNeighborsClassifier recall: 0.9741379310344828
KNeighborsClassifier precision: 0.7290322580645161

Confusion matrix:
 [[ 18  84]
 [  6 226]]


## Нормализованные данные

In [ ]:
# делим данными на тестовые и тренировочные с помощью метода train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

# создаем модель
knn = KNeighborsClassifier(n_neighbors = 20)

# обучаем модель
knn.fit(X_train, y_train)

# тестируем модель
y_predicted_knn_norm = knn.predict(X_test)

# оценка
knn_accuracy_norm = accuracy_score(y_test,y_predicted_knn_norm)
knn_recall_norm =  recall_score(y_test,y_predicted_knn_norm)
knn_precision_norm =  precision_score(y_test,y_predicted_knn_norm)

# результат на нормализованных данных оказался немного хуже чем на обычных данных 
print("KNN accuracy:", knn_accuracy_norm)
print("KNN recall:", knn_recall_norm)
print("KNN precision:", knn_precision_norm)

# матрица ошибок модели
print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predicted_knn_norm))

KNeighborsClassifier accuracy: 0.718562874251497
KNeighborsClassifier recall: 0.9525862068965517
KNeighborsClassifier precision: 0.7269736842105263

Confusion matrix:
 [[ 19  83]
 [ 11 221]]


## Стандартизованные данные

In [ ]:
# делим данными на тестовые и тренировочные с помощью метода train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

# создаем модель
knn = KNeighborsClassifier(n_neighbors = 20)

# обучаем модель
knn.fit(X_train, y_train)

# тестируем модель
y_predicted_knn_st = knn.predict(X_test)

# оценка
knn_accuracy_st = accuracy_score(y_test,y_predicted_knn_st)
knn_recall_st =  recall_score(y_test,y_predicted_knn_st)
knn_precision_st =  precision_score(y_test,y_predicted_knn_st)

# результат на стандартизированных данных такой же как и на обычных
print("KNN accuracy:", knn_accuracy_st)
print("KNN recall:", knn_recall_st)
print("KNN precision:", knn_precision_st)

# матрица ошибок модели
print('\nConfusion matrix:\n', confusion_matrix(y_test, y_predicted_knn_st))

KNeighborsClassifier accuracy: 0.7395209580838323
KNeighborsClassifier recall: 0.9568965517241379
KNeighborsClassifier precision: 0.7424749163879598

Confusion matrix:
 [[ 25  77]
 [ 10 222]]


## Кроссвалидация

In [ ]:
kf = KFold()

# создаем модель
kneighbours = KNeighborsClassifier(n_neighbors = 20)

# оценка
kneighbours_crs_accuracy = cross_val_score(estimator=knn, X=X, y=y, cv=kf, scoring='accuracy').mean()
kneighbours_crs_recall = cross_val_score(estimator=knn, X=X, y=y, cv=kf, scoring='recall').mean()
kneighbours_crs_precision = cross_val_score(estimator=knn, X=X, y=y, cv=kf, scoring='precision').mean()

# оценки модели с кроссвалидацией данных также вышли хорошими
print('KNN cross_val_score accuracy: ', kneighbours_crs_accuracy)
print('KNN cross_val_score recall: ', kneighbours_crs_recall)
print('KNN cross_val_score precision: ', kneighbours_crs_precision)

KNeighborsClassifier cross_val_score accuracy:  0.72
KNeighborsClassifier cross_val_score recall:  0.9585819404064159
KNeighborsClassifier cross_val_score precision:  0.7281147204824892
